<a href="https://colab.research.google.com/github/DaryaTereshchenko/AdClick/blob/main/ad_click.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.7

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pprint import pprint
import re

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import IntegerLookup, Concatenate, Flatten


In [7]:
# load data
from urllib.request import urlretrieve
import os

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

d10m_url = "https://home.ipipan.waw.pl/sj/TIB_PAN_Adv/AdClick/D10M.tsv.gz"
download(d10m_url,'D10M.tsv.gz')
print("All the files are downloaded")


All the files are downloaded


In [28]:
d5m_url = "https://home.ipipan.waw.pl/sj/TIB_PAN_Adv/AdClick/D5M_test_x.tsv.gz"
download(d5m_url,'D5M_test_x.tsv.gz')
print("All the files are downloaded")

All the files are downloaded


In [9]:
SELECT_COLUMNS = ["Position", "Age", "Depth", "Gender", "AdvertiserId", "AdDescription_tokens", "AdKeyword_tokens", "Query_tokens", "Click"]
DEFAULTS = [tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.string, tf.string, tf.string] 

In [29]:
test_data = tf.data.experimental.make_csv_dataset("D5M_test_x.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                           num_epochs=1,
                                           column_defaults = DEFAULTS,
                                           batch_size=1024,  
                                           label_name="Click",
                                           select_columns= SELECT_COLUMNS,
                                           shuffle=False, ignore_errors=True)

In [ ]:
pprint(test_data.element_spec)

In [12]:
ds = tf.data.experimental.make_csv_dataset("D10M.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                           num_epochs=1,
                                           batch_size=1024,  
                                           label_name="Click",
                                           select_columns= SELECT_COLUMNS,
                                           shuffle=False)


In [13]:
def get_dataset_partitions_tf(ds, ds_size=10000000, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [14]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(ds)

In [15]:
train = train_ds.unbatch()

needed_vocab = {"AdKeyword_tokens": [], "AdDescription_tokens": [], "Query_tokens": []}

for dic in train.as_numpy_iterator():
  for k, v in dic[0].items():
    if k in needed_vocab.keys():
      needed_vocab[k].append(v)

In [16]:
def transform_to_list(parameter):
  list_of_tokens = list(map(lambda x: x.replace(b"|", b" ").split(b" "), parameter))
  flatten_list = [item for subl in list_of_tokens for item in subl]
  return np.asarray(flatten_list)


In [18]:
keyword_tokens = transform_to_list(needed_vocab["AdKeyword_tokens"])
description_tokens = transform_to_list(needed_vocab["AdDescription_tokens"])
query_tokens = transform_to_list(needed_vocab["Query_tokens"])
total_vocab = np.unique(np.concatenate((keyword_tokens, description_tokens, query_tokens), axis=0))

In [19]:
maxlen = 128
max_features = 221395 # max word number

In [20]:
def split_on_slash(input_data):
  return tf.strings.regex_replace(input_data, "\|", " ")

In [ ]:
"""
  AUC SCORE 75.56%
"""

def build_model1():
  inputs = {
      "Position": Input(shape=(), dtype=tf.int32),
      "Age": Input(shape=(), dtype=tf.int32),
      "Depth": Input(shape=(), dtype=tf.int32),
      "Gender": Input(shape=(), dtype=tf.int32),
      "AdvertiserId": Input(shape=(), dtype=tf.int32),
      "Query_tokens": Input(shape=(), dtype=tf.string),
      "AdDescription_tokens": Input(shape=(), dtype=tf.string),
      "AdKeyword_tokens": Input(shape=(), dtype=tf.string)}

  # Integer part
  input_age = IntegerLookup(vocabulary=[1,2,3,4,5,6], output_mode="one_hot", num_oov_indices=0)(inputs["Age"])
  ci_p = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Position"])
  ci_d = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Depth"])
  ci_g = IntegerLookup(vocabulary=[0,1,2], output_mode="one_hot", num_oov_indices=0)(inputs["Gender"])
  encoded = Concatenate()([ci_p, input_age, ci_d, ci_g])
  layer = tf.keras.layers.Hashing(num_bins=128, output_mode="one_hot")(inputs["AdvertiserId"])
  conc = tf.keras.layers.concatenate([layer, encoded])
  num = Dropout(rate=0.3)(conc)
  num = Dense(128)(num)
  num = Flatten()(num)

  # String part
  description = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["AdDescription_tokens"])
  x = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(description)

  query = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["Query_tokens"])
  y = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(query)

  keywords = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["AdKeyword_tokens"])
  o = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(keywords)
  
  c = Concatenate()([x, y, o])
  t = Dropout(rate=0.3)(c)
  t = Dense(128)(t)
  t = Flatten()(t)

  z = Concatenate()([t, num])
  z = tf.keras.layers.Dense(64, activation='relu')(z)
  z = tf.keras.layers.BatchNormalization()(z)
  z = tf.keras.layers.Dense(1, activation='sigmoid')(z)
  

  model = Model(inputs=inputs, outputs=z)
  model.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          metrics=["AUC"], run_eagerly=True)
  return model

In [21]:
"""
  AUC SCORE 77.48%
"""
def build_model2():
  inputs = {
      "Position": Input(shape=(), dtype=tf.int32),
      "Age": Input(shape=(), dtype=tf.int32),
      "Depth": Input(shape=(), dtype=tf.int32),
      "Gender": Input(shape=(), dtype=tf.int32),
      "AdvertiserId": Input(shape=(), dtype=tf.int32),
      "AdDescription_tokens": Input(shape=(), dtype=tf.string),
      "Query_tokens": Input(shape=(), dtype=tf.string),
      "AdKeyword_tokens": Input(shape=(), dtype=tf.string)}

  # Integer part
  input_age = IntegerLookup(vocabulary=[1,2,3,4,5,6], output_mode="one_hot", num_oov_indices=0)(inputs["Age"])
  ci_p = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Position"])
  ci_d = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Depth"])
  ci_g = IntegerLookup(vocabulary=[0,1,2], output_mode="one_hot", num_oov_indices=0)(inputs["Gender"])
  encoded = Concatenate()([ci_p, input_age, ci_d, ci_g])
  layer = tf.keras.layers.Hashing(num_bins=1024, output_mode="one_hot")(inputs["AdvertiserId"])

  conc = tf.keras.layers.concatenate([layer, encoded])
  num = Dropout(rate=0.3)(conc)
  num = Dense(128)(num)
  num = Flatten()(num)

# String part
  description = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["AdDescription_tokens"])
  x = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(description)
  
  title = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["Query_tokens"])
  y = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(title)

  keywords = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode="int", pad_to_max_tokens=maxlen, standardize=split_on_slash, output_sequence_length=maxlen, vocabulary=total_vocab)(inputs["AdKeyword_tokens"])
  o = tf.keras.layers.Embedding(max_features, output_dim=10, input_length=maxlen)(keywords)

  c = Concatenate()([x, y, o])
  c = tf.keras.layers.SpatialDropout1D(0.3)(c)
  c = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(c)
  c = tf.keras.layers.GlobalAveragePooling1D()(c)
  t = Dropout(rate=0.3)(c)
  t = Flatten()(t)

  z = Concatenate()([t, num])
  z = tf.keras.layers.Dense(64, activation='relu')(z)
  z = tf.keras.layers.BatchNormalization()(z)
  z = tf.keras.layers.Dense(1, activation='sigmoid')(z)
  

  model = Model(inputs=inputs, outputs=z)
  model.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          metrics=["AUC"], run_eagerly=True)
  return model

In [ ]:
model = build_model2()
model.summary()

In [24]:
model.fit(train_ds, epochs=12, verbose=1, validation_data=val_ds)

Epoch 1/12
9766/9766 [==============================] - 1419s 127ms/step - loss: 0.1806 - auc: 0.6847
Epoch 2/12
9766/9766 [==============================] - 1241s 110ms/step - loss: 0.1724 - auc: 0.7253
Epoch 3/12
9766/9766 [==============================] - 1252s 110ms/step - loss: 0.1706 - auc: 0.7377
Epoch 4/12
9766/9766 [==============================] - 1250s 110ms/step - loss: 0.1693 - auc: 0.7459
Epoch 5/12
9766/9766 [==============================] - 1250s 110ms/step - loss: 0.1683 - auc: 0.7523
Epoch 6/12
9766/9766 [==============================] - 1242s 109ms/step - loss: 0.1674 - auc: 0.7574
Epoch 7/12
9766/9766 [==============================] - 1227s 108ms/step - loss: 0.1666 - auc: 0.7617
Epoch 8/12
9766/9766 [==============================] - 1228s 109ms/step - loss: 0.1660 - auc: 0.7653
Epoch 9/12
9766/9766 [==============================] - 1230s 109ms/step - loss: 0.1653 - auc: 0.7681
Epoch 10/12
9766/9766 [==============================] - 1226s 109ms/step - loss: 

In [30]:
predict = model.predict(test_data)

4883/4883 [==============================] - 291s 60ms/step


In [31]:
with open('drive/MyDrive/results_Stetsenko.txt', 'w') as f:
  for i in predict:
    f.write(str(i))
    f.write('\n')
f.close

<function TextIOWrapper.close()>